<!-- Assignment X- WS 2020 -->

# Regular-, Initial- &amp; Normalisation (11 points)

This notebook contains the bonus assignment for the exercises in Deep Learning and Neural Nets 1.
It provides a skeleton, i.e. code with gaps, that will be filled out by you in different exercises.
All exercise descriptions are visually annotated by a vertical bar on the left and some extra indentation,
unless you already messed with your jupyter notebook configuration.
Any questions that are not part of the exercise statement do not need to be answered,
but should rather be interpreted as triggers to guide your thought process.

**Note**: The cells in the introductory part (before the first subtitle)
perform all necessary imports and provide utility function that should work without problems.
Please, do not alter this code or add extra import statements in your submission, unless it is explicitly requested!

<span style="color:#d95c4c">**IMPORTANT:**</span> Please, change the name of your submission file so that it contains your student ID!

In this assignment, the goal is to get familiar with some tools that can help to speed up the training process of neural networks. **Regularisation** is a technique that can be used to avoid overfitting. Knowing what kind of **initialisation** to use in what context is often important to assure fast learning. **Normalisation** is a tool that tackles the problem of drifting distributions that pops up in very deep networks and hinders learning.

In [1]:
import numpy as np

from nnumpy import Module, Flatten
from nnumpy.testing import gradient_check

rng = np.random.default_rng(1856)

In [2]:
def initialiser(fn):
    """ 
    Function decorator for initialisation functions that
    enables initialisation of multiple weight arrays at once. 
    """
    
    def init_wrapper(*parameters, **kwargs):
        for par in parameters:
            par[:] = fn(par.shape, **kwargs)
            par.zero_grad()
    
    init_wrapper.__name__ = fn.__name__ + "_init"
    init_wrapper.__doc__ = fn.__doc__
    return init_wrapper

## Regularisation

Neural networks are infamously prone to overfitting. Just as with any machine learning model, overfitting can relatively easily be detected by monitoring the learning curves on training and validation sets. In order to counter these effects, you can use regularisation techniques. 

![learning curves](https://d2l.ai/_images/capacity_vs_error.svg)

One possibility is to use well-known approaches from regression: e.g. $L_1$ or $L_2$ regularisation, which are also known as *LASSO*, resp. *ridge* regression. Also simply interrupting the learning before the overfitting occurs can prevent overfitting models. These are only a few examples, but most regularisation techniques are not exlusive to neural networks. However, there is one NN-exclusive approach that is very commonly used: **Dropout**.

### Exercise 1: Dropout (3 Points)

Dropout is a simple, but very effective regularisation technique that can be added practically anywhere in a network. The idea of dropout is to randomly disable a few neurons during training. During inference all neurons are used. Since this would lead to a shift in distribution of the pre-activations in the next layer (training vs inference), the neurons are scaled down during evaluation so that the distributions during inference and training are approximately the same. In order to avoid the need to change the network during evaluation, it is also possible to scale up the activations during training. This specific change in implementation is often referred to as *inverted* dropout.

> Implement the forward and backward pass of an **inverted dropout** module.

**Hint:** use the `Module` attribute `predicting` to check whether you are in prediction or training or mode.

In [3]:
class Dropout(Module):
    """ NNumpy implementation of (inverted) dropout. """

    def __init__(self, rate: float = .5, seed: int = None):
        """
        Parameters
        ----------
        rate : float, optional
            The percentage of neurons to be dropped.
        seed : int, optional
            Seed for the pseudo random generator.
        """
        super().__init__()
        if rate < 0. or rate > 1.:
            raise ValueError("dropout rate should be between zero and one")

        self.rate = float(rate)
        self.rng = np.random.default_rng(seed)

    def compute_outputs(self, x):
        # use self.rng, instead of np.random to generate random numbers with the module seed
        if self.predicting:
            multiplier = np.ones(x.shape)
            # raise NotImplementedError("TODO: implement prediction mode of Dropout.compute_outputs!")
        else:
            multiplier = self.rng.binomial(1, 1-self.rate, x.shape) / (1-self.rate)  # (self.rng.random(x.shape) < self.rate)
            #  raise NotImplementedError("TODO: implement training mode of Dropout.compute_outputs!")
        return x * multiplier, multiplier

    def compute_grads(self, grads, multiplier):
        dx = grads * multiplier
        return dx
        raise NotImplementedError("TODO: implement Dropout.compute_grads!")


dropout = Dropout()
do_check = gradient_check(dropout, rng.standard_normal(size=(1, 11, 13)), debug=True)
print("gradient check for Dropout:", "passed" if do_check else "failed")

gradient check for Dropout: passed


In [4]:
dropout = Dropout()
do_check = gradient_check(dropout, rng.standard_normal(size=(1, 11, 13)), debug=True)
print("gradient check for Dropout:", "passed" if do_check else "failed")

gradient check for Dropout: passed


## Initialisation

A good initialisation has proven to be very important to learn deep neural networks. Although this can be considered as a well-known fact, it is astonishing how often initialisation is ignored. Since simply initialising all parameters with some constant does not work, the initial values are generally small, randomly generated numbers. There are different distributions to sample these values from, however.

### Exercise 2: Xavier Glorot (3 Points)

When generating random values, there are different choices for the distribution to draw numbers from. The uniform or Gaussian (a.k.a. normal) distributions are most common for initialising the parameters of a neural network. After all, these are simple distributions that can easily be centred around zero.

Apart from centring the initial parameters around zero, it is also helpful to make sure that the weights have a specific amount of variance. Xavier Glorot proposed to use the reciprocal of the average of fan-in and fan-out, i.e. $\frac{2}{\text{fan-in} + \text{fan-out}}$, for the variance. Here, *fan-in* and *fan-out* are the number of incoming connections per output neuron and number of outgoing connections per input neuron, respectively.

Note, however, that this proposal only holds for identity and $\tanh$ activation functions. When using different activation functions, the variance of the initial parameters need to be scaled correspondingly. This can be done by means of a linear *gain* factor that accounts for the effect of the activation functions.

 > Implement the `glorot_uniform` function so that it produces initial weights for a parameter with given shape according to the proposal from Xavier Glorot. Make sure to make use of the seed for the initialisation, as well as the `gain` parameter.
 
**Hint:** Think carefully about the number of connections in convolutional layers.

In [5]:
@initialiser
def glorot_uniform(shape, gain: float = 1., seed: int = None):
    """
    Initialise parameter cf. Glorot, using a uniform distribution.
    
    Parameters
    ----------
    shape : tuple
        The shape of the parameter to be initialised.
    gain : float, optional
        Multiplier for the variance of the initialisation.
    seed : int, optional
        Seed for generating pseudo random numbers.
        
    Returns
    -------
    values: ndarray
        Numpy array with the initial weight values
        with dimensions as specified by `shape`.
    """
    rng = np.random.default_rng(seed)
    values = gain * rng.uniform(size=shape) * (2/(np.sum(shape)))
    var = gain * 2/np.sum(shape)
    values = var * rng.randn(size=shape)
    return values
    raise NotImplementedError("TODO: implement glorot_uniform function!")

In [6]:
# you can include your modules here for the sanity check
# e.g. from nnumpy import Linear, Conv2D
from nnumpy.nn_modules import Conv2D, Linear

In [7]:
# sanity check
try:
    fc = Linear(120, 80)
    conv = Conv2D(4, 16, (5, 5))
    glorot_uniform(fc.w, conv.w, seed=1806)
    print(f"linear var: {fc.w.var():.2e}")
    print(f"  conv var: {conv.w.var():.2e}")
except NameError:
    print("no sanity check...")

AttributeError: 'numpy.random._generator.Generator' object has no attribute 'randn'

## Normalisation

Whereas initialisation ensures proper variance propagation through the network when learning starts, it does not ensure that the weights keep these properties after some updates. To ensure a steady flow of information through the network, normalisation techniques were introduced. The idea of normalisation is to normalise either the activations or pre-activations. This can be done explicitly, using techniques like *Batch* or *Layer Normalisation*, or more implicitly, e.g. *weight normalisation* or using *self-normalising networks*.

### Exercise 5: Batch Normalisation (5 Points)

Batch Normalisation (or *batch norm* for short) has empirically proven to be a very useful technique for improving the performance of neural networks. It is not quite clear why it works so well, but there is some form of consensus that it acts as a regulariser and improves gradient flow in the network. 

The core principle of batch norm is to subtract the mean and divide by the standard deviation of the data, computed over the samples in one batch. Each neuron is normalised individually, so that all neurons have zero mean and unit variance. Batch norm also uses parameters $\gamma$ and $\beta$ to scale, resp. shift the normalised signal. Note that, since batch norm relies on batch statistics, it requires a large batch size to work properly!

During inference, it is not uncommon to want a prediction for a single sample. Therefore, you generally do not want to use the mean computed during inference. Therefore, batch norm tracks the statistics of the data during training using a [moving average](https://en.wikipedia.org/wiki/Moving_average). During evaluation, these tracked statistics, i.e. the statistics of the training data, are used to normalise the previously unseen samples.

 > Implement the forward and backward pass of the batch normalisation module. Use a simple moving average for tracking the statistics.
 
**Hint:** You can track the statistics in attributes of the module.

In [8]:
class BatchNormalisation(Module):
    """ NNumpy implementation of batch normalisation. """

    def __init__(self, dims: tuple, eps: float = 1e-8):
        """
        Parameters
        ----------
        dims : tuple of ints
            The shape of the incoming signal (without batch dimension).
        eps : float, optional
            Small value for numerical stability.
        """
        super().__init__()
        self.dims = tuple(dims)
        self.eps = float(eps)

        self.gamma = self.register_parameter('gamma', np.ones(self.dims))
        self.beta = self.register_parameter('beta', np.zeros(self.dims))

        self.running_count = 0
        self.running_stats = np.zeros((2,) + self.dims)

    def compute_outputs(self, x):
        if self.predicting:
            self.running_stats[0] = np.zeros(x.shape[1:])
            self.running_stats[1] = np.ones(x.shape[1:])
            # pass
            # raise NotImplementedError("TODO: implement prediction mode of BatchNormalisation.compute_outputs!")
        else:
            x_mean = np.mean(x, axis=0)
            x_var = np.var(x, axis=0)
            self.running_stats[0] = x_mean
            self.running_stats[1] = x_var
            # raise NotImplementedError("TODO: implement training mode of BatchNormalisation.compute_outputs!")

        x_norm = (x - self.running_stats[0])/np.sqrt(self.running_stats[1] + self.eps)
        out = self.gamma * x_norm + self.beta
        self.running_count = x.shape[0]
        cache = x_norm, x
        return out, cache

    def compute_grads(self, grads, cache):
        x_norm, x = cache
        N = self.running_count #  grads.shape[0]

        dx_norm = grads * self.gamma  # / np.sqrt(self.running_stats[1] + self.eps)
        self.beta.grad = grads.sum(axis=0)

        self.gamma.grad = (grads*x_norm).sum(axis=0)

        # dx_center = dx_norm / (np.sqrt(self.running_stats[1] + self.eps))
        # dmean = -(dx_center.sum(axis=0) + 2/N * (x - self.running_stats[0]).sum(axis=0))
        # dstd = (dx_norm * (x - self.running_stats[0]) / (self.running_stats[1] + self.eps)).sum(axis=0)
        # dvar = dstd / 2 / (np.sqrt(self.running_stats[1] + self.eps))
        # dx = dx_center + (dmean + dvar * 2 * (x - self.running_stats[0])) / N

        dx = (1./N) / np.sqrt(self.running_stats[1] + self.eps) * (N * dx_norm - dx_norm.sum(axis=0) - x_norm * (dx_norm * x_norm).sum(axis=0))

        return dx
        raise NotImplementedError("TODO: implement BatchNormalisation.compute_grads!")

In [9]:
# sanity check
x = rng.uniform(0, 3, size=(7, 3, 5))
batch_norm = BatchNormalisation(x.shape[1:])
s = batch_norm(x)
print(f"mean: {s.mean()}, var: {s.var()}")

mean: 4.229421046191073e-18, var: 0.9999999844652488


In [10]:
# sanity check
bn_check = gradient_check(batch_norm, x, debug=True)
print("gradient check for BatchNormalisation:", "passed" if bn_check else "failed")

gradient check for BatchNormalisation: passed
